In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc", cache_dir='./cache')


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [2]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [3]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

/home/cpadwick/.cache/pypoetry/virtualenvs/hf-llm-course--JTTk5sz-py3.12/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


In [4]:
from huggingface_hub import login
login()

In [5]:
from transformers import AutoModelForSequenceClassification, AutoConfig

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import torch

In [7]:
torch.cuda.is_available()

True

In [8]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [9]:
trainer.train()

* Trackio project initialized: huggingface
* Trackio metrics will be synced to Hugging Face Dataset: cgpadwick2020/trackio-dataset
* Found existing space: https://huggingface.co/spaces/cgpadwick2020/trackio
* View dashboard by going to: https://cgpadwick2020-trackio.hf.space/


* Created new run: cgpadwick2020-1762832804


Step,Training Loss
500,0.528100
1000,0.303400


* Run finished. Uploading logs to Trackio (please wait...)


KeyboardInterrupt: 

In [10]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [11]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [12]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8700980392156863, 'f1': 0.9084628670120898}

In [13]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
training_args = TrainingArguments("test-trainer", eval_strategy="epoch", num_train_epochs=5)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
trainer.train()

* Created new run: cgpadwick2020-1762833097


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.424971,0.833333,0.870722
2,0.549400,0.455692,0.852941,0.900662
3,0.351200,0.611333,0.850490,0.890877
4,0.197400,0.761052,0.852941,0.898990
5,0.086200,0.746253,0.865196,0.905983


* Run finished. Uploading logs to Trackio (please wait...)


KeyboardInterrupt: 

In [16]:
training_args = TrainingArguments("test-trainer", 
                                  eval_strategy="epoch",
                                  num_train_epochs=3,
                                 fp16=True)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.404944,0.821078,0.868468
2,0.524300,0.432226,0.865196,0.904348
3,0.294800,0.594649,0.865196,0.906303


TrainOutput(global_step=1377, training_loss=0.3420354805740656, metrics={'train_runtime': 102.6814, 'train_samples_per_second': 107.166, 'train_steps_per_second': 13.41, 'total_flos': 405114969714960.0, 'train_loss': 0.3420354805740656, 'epoch': 3.0})

In [34]:
training_args = TrainingArguments("test-trainer", 
                                  eval_strategy="epoch",
                                  num_train_epochs=3,
                                  fp16=True,
                                 lr_scheduler_type="cosine")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.520186,0.757353,0.842105
2,0.590300,0.518858,0.816176,0.873524
3,0.415000,0.688941,0.830882,0.882051


TrainOutput(global_step=1377, training_loss=0.4446430580288756, metrics={'train_runtime': 90.9402, 'train_samples_per_second': 121.003, 'train_steps_per_second': 15.142, 'total_flos': 405114969714960.0, 'train_loss': 0.4446430580288756, 'epoch': 3.0})